## 实验二：搭建VGG16神经网络实现图像分类

**实验目的**：
    掌握卷积神经网络的设计原理，能够独立构建卷积神经网络，深入了解基本算子的正向传播及反向传播原理，能够使用 Python 语言构建 VGG16 网络模型来对给定的输入图像进行分类，能够独立编写基本算子的正向传播及反向传播代码。

**实验内容**：
* Convolution算子的正向传播及反向传播代码实现 
* MaxPool算子的正向传播及反向传播代码实现 
* VGG16网络搭建

In [1]:
# 导入相关依赖库
import time
import numpy as np
from numba import jit
import cv2

### 1.Convolution算子实现

卷积算子的实现如下所示，其中定义了以下成员函数：

* 算子初始化：需要定义卷积算子的超参数，包括输入张量的通道数$C_{in}$，输出张量的通道数$C_{out}$，卷积核的尺寸$K$，边界扩充大小$P$，卷积步长$S$。此外还需要定义输入张量的形状，用于反向传播。
* 权重初始化：卷积算子的参数包括权重和偏置。通常使用高斯随机数来初始化权重，将偏置值均设为0。
* 正向传播计算：根据公式进行卷积算子正向传播的计算，首先对输入张量`inputs`进行边界填充得到`inputs_pad`，在填充后的张量`inputs_pad`上滑动卷积窗口。
* 反向传播计算：根据公式进行卷积算子反向传播的计算（因为不涉及参数更新，故忽略计算偏置的梯度）。
* 参数加载：通过输入指定卷积算子的权重和偏置参数。

正向传播公式：
$$Y(n,c,h,w) = \sum_{k=0}^{C_{in}-1} \sum_{i=0}^{K-1} \sum_{j=0}^{K-1} Weight(c,k,i,j)*X_{pad}(n,k,h_{s}+i,w_{s}+j) + Bias(c) \\
n \in [0, N), c \in [0, C_{out}), h \in [0,H_{out}),w\in [0,W_{out}) \\
h_{s} = h*S, w_s = w*S$$

反向传播公式：
$$\sum_{c_{in}=0}^{C_{in}-1}\sum_{i=0}^{K-1}\sum_{j=0}^{K-1} \nabla_{in_{pad}}(n,c_{in},h_s+i,w_s+i) = \sum_{c_{out}=0}^{C_{out}-1} \nabla_{out}(n,c_{out},h,w)*Weight(c_{out},c_{in},i,j) \\
n \in [0, N), h \in [0,H_{out}),w\in [0,W_{out}) \\
h_{s} = h*S, w_s = w*S$$

In [2]:
class ConvolutionLayer(object):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0, stride=1):
        # 输入通道数
        self.in_channels = in_channels
        # 输出通道数
        self.out_channels = out_channels
        # 卷积核尺寸
        self.kernel_size = kernel_size
        # 步长
        self.stride = stride
        # 填充长度
        self.padding = padding

        # 卷积核权重
        self.weight = np.random.normal(loc=0.0, scale=0.01,
                                       size=(self.out_channels, self.in_channels,
                                             self.kernel_size, self.kernel_size))
        # 卷积核偏置
        self.bias = np.zeros([self.out_channels])

        # 输入张量的形状，用于反向传播
        self.input_shape = None

    def forward(self, inputs):
        # 记录输入张量的形状，inputs: (N,C,H,W)
        self.input_shape = inputs.shape
        batch, channel, height, width = inputs.shape

        # 获取输入张量填充后的宽高
        pad_height = height + self.padding * 2
        pad_width = width + self.padding * 2

        # 将输入张量进行填充
        inputs_pad = np.zeros((batch, channel, pad_height, pad_width), dtype=inputs.dtype)
        inputs_pad[:, :, self.padding:height + self.padding, self.padding:width + self.padding] = inputs

        # 获取输出张量的宽高，并构建输出张量
        out_height = int((pad_height - self.kernel_size) / self.stride + 1)
        out_width = int((pad_width - self.kernel_size) / self.stride + 1)
        outputs = np.zeros((batch, self.out_channels, out_height, out_width), dtype=inputs.dtype)
        
        # 正向传播
        outputs = self._conv(inputs_pad, outputs, self.weight, self.bias, self.kernel_size, self.stride)
        return outputs

    def backward(self, out_grad):
        # 获得输入张量，填充后输入张量，输出张量的形状
        batch, channel, height, width = self.input_shape
        _, out_channel, out_height, out_width = out_grad.shape
        pad_height = height + self.padding * 2
        pad_width = width + self.padding * 2

        # 构建填充输入张量的梯度
        in_grad = np.zeros((batch, channel, pad_height, pad_width))

        # 反向传播
        in_grad = self._conv_back(out_grad, in_grad, self.weight, self.kernel_size, self.stride)
        
        # 返回输入张量梯度
        in_grad = in_grad[:, :, self.padding:height + self.padding, self.padding:width + self.padding]
        return in_grad

    def load_params(self, weight, bias):
        assert self.weight.shape == weight.shape
        assert self.bias.shape == bias.shape
        self.weight = weight
        self.bias = bias

    @staticmethod
    @jit(nopython=True)     # 可以将python函数编译为机器代码的JIT编译器，可以极大的加速for循环的运行速度
    def _conv(inputs_pad, outputs, weight, bias, kernel_size, stride):
        # TODO：根据公式编写下列代码 请用for循环实现
        in_channels = inputs_pad.shape[1]
        batch, out_channels, out_height, out_width = outputs.shape
        for n in range(batch):
            for c in range(out_channels):
                for h in range(out_height):
                    for w in range(out_width):
                        hs, ws = h * stride, w * stride
                        val = 0
                        for k in range(in_channels):
                            for i in range(kernel_size):
                                for j in range(kernel_size):
                                    val += weight[c, k, i, j] * inputs_pad[n, k, hs+i, ws+j]
                        val += bias[c]
                        outputs[n, c, h, w] = val
        return outputs

    @staticmethod
    @jit(nopython=True)
    def _conv_back(out_grad, in_grad, weight, kernel_size, stride):
        # TODO：根据公式编写下列代码 请用for循环实现
        in_channels = in_grad.shape[1]
        batch, out_channel, out_height, out_width = out_grad.shape
        for n in range(batch):
            for h in range(out_height):
                for w in range(out_width):
                    hs, ws = h * stride, w * stride
                    for c_in in range(in_channels):
                        for i in range(kernel_size):
                            for j in range(kernel_size):
                                val = 0
                                for c_out in range(out_channel):
                                    val += out_grad[n, c_out, h, w] * weight[c_out, c_in, i, j]
                                in_grad[n, c_in, hs + i, ws + j] += val
        return in_grad

### 2.MaxPool算子实现

最大池化算子的实现如下所示，其中定义了以下成员函数：

* 算子初始化：需要定义最大池化算子的超参数，包括池化核的尺寸$K$，池化步长$S$。此外初始化了用于反向传播的池化索引，输入张量的形状和输出张量的形状。
* 正向传播计算：根据公式进行池化算子正向传播的计算。
* 反向传播计算：根据公式进行池化算子反向传播的计算。在正向传播时，已经记录了池化索引，在反向传播时，只需将池化索引映射回输入张量的位置，将梯度带过去即可，其余位置置为0。

正向传播公式：
$$Y(n,c,h,w) = \mathop{max}\limits_{m=0,..K-1} \space \mathop{max}\limits_{n=0,..K-1} X(n, c, h_s+m, w_s+n)\\
n \in [0, N), c \in [0, C), h \in [0,H_{out}),w\in [0,W_{out}) \\
h_{s} = h*S, w_s = w*S$$

反向传播公式：
$$\nabla_{in}(n,c,h_s:h_s+K,w_s:w_s+K)[i_{index},j_{index}]=\nabla_{out}(n,c,h,w)\\
n \in [0, N),c \in [0, C), h \in [0,H_{out}),w\in [0,W_{out}) \\
h_{s} = h*S, w_s = w*S \\
$$
其中 $i_{index}$ 和 $j_{index}$ 在正向传播中记录下的索引值。

In [3]:
class MaxPoolLayer(object):
    def __init__(self, kernel_size=2, stride=2):
        # 池化核大小
        self.kernel_size = kernel_size
        # 步长
        self.stride = stride
        # 池化索引，用于反向传播
        self.argidx = None
        # 输入张量形状
        self.input_shape = None
        # 输出张量形状
        self.output_shape = None

    def forward(self, inputs):
        # inputs: (N,C,H,W)
        batch, channel, height, width = inputs.shape

        # 获取输出张量的宽高，并构建输出张量
        out_height = int((height - self.kernel_size) / self.stride + 1)
        out_width = int((width - self.kernel_size) / self.stride + 1)
        outputs = np.zeros((batch, channel, out_height, out_width), dtype=inputs.dtype)

        # 记录输入张量和输出张量的形状，并初始化池化索引
        self.input_shape = inputs.shape
        self.output_shape = outputs.shape
        self.argidx = np.zeros_like(outputs, dtype=np.int32)

        # 正向传播
        outputs, self.argidx = self._pool(outputs, inputs, self.argidx, self.kernel_size, self.stride)
        return outputs

    def backward(self, out_grad):
        # 构建输入梯度
        in_grad = np.zeros(self.input_shape)

        # 反向传播
        in_grad = self._pool_back(out_grad, in_grad , self.argidx, self.kernel_size, self.stride)
        return in_grad

    @staticmethod
    @jit(nopython=True)
    def _pool(outputs, inputs, argidx, kernel_size, stride):
        # TODO：根据公式编写下列代码 请用for循环实现
        batch, channel, out_height, out_width = outputs.shape
        for n in range(batch):
            for c in range(channel):
                for h in range(out_height):
                    for w in range(out_width):
                        hs, ws = h*stride, w*stride
                        vector = inputs[n, c, hs:hs+kernel_size, ws:ws+kernel_size]
                        max_value = vector[0][0]
                        for i in range(kernel_size):
                            for j in range(kernel_size):
                                if vector[i, j] > max_value:
                                    max_value = vector[i, j]
                                    # 记录当前索引
                                    argidx[n, c, h, w] = i * kernel_size + j
                        outputs[n, c, h, w] = max_value
        return outputs, argidx

    @staticmethod
    @jit(nopython=True)
    def _pool_back(out_grad, in_grad, argidx, kernel_size, stride):
        # TODO：根据公式编写下列代码 请用for循环实现
        batch, channel, out_height, out_width = out_grad.shape
        for n in range(batch):
            for c in range(channel):
                for h in range(out_height):
                    for w in range(out_width):
                        hs, ws = h*stride, w*stride
                        # 将索引逆向转换至卷积核位置
                        i = argidx[n, c, h, w] // kernel_size
                        j = argidx[n, c, h, w] % kernel_size
                        in_grad[n, c, hs: hs+kernel_size, ws: ws+kernel_size][i, j] = out_grad[n, c, h, w]

        return in_grad


### 3. 扁平化算子实现

扁平化算子的实现如下所示，其中定义了以下成员函数：

* 正向传播计算：进行Flatten算子正向传播的计算。将四维张量$(N,C,H,W)$，扁平化至二维$(N,C*H*W)$
* 反向传播计算：进行Flatten算子反向传播的计算。将二维梯度$(N,C*H*W)$映射回四维梯度$(N,C,H,W)$即可。

In [4]:
class FlattenLayer(object):
    def __init__(self):
        self.input_shape = None

    def forward(self, inputs):
        # inputs: (N,C,H,W) -> (N, C*H*W)
        self.input_shape = inputs.shape
        batch, channel, height, width = inputs.shape
        return inputs.reshape((batch, channel * height * width))

    def backward(self, out_grad):
        return out_grad.reshape(self.input_shape)

### 4.搭建VGG16网络

In [5]:
# 导入算子
from layer import ReluLayer, FullyConnectLayer, CrossEntropy

class VGG16(object):
    def __init__(self, num_classes=4):
        # TODO 根据网络图搭建VGG16模型
        self.layer1_conv1 = ConvolutionLayer(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.layer1_relu1 = ReluLayer()
        self.layer1_conv2 = ConvolutionLayer(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.layer1_relu2 = ReluLayer()
        self.layer1_maxpool = MaxPoolLayer(kernel_size=2, stride=2)

        self.layer2_conv1 = ConvolutionLayer(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.layer2_relu1 = ReluLayer()
        self.layer2_conv2 = ConvolutionLayer(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.layer2_relu2 = ReluLayer()
        self.layer2_maxpool = MaxPoolLayer(kernel_size=2, stride=2)

        self.layer3_conv1 = ConvolutionLayer(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.layer3_relu1 = ReluLayer()
        self.layer3_conv2 = ConvolutionLayer(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.layer3_relu2 = ReluLayer()
        self.layer3_conv3 = ConvolutionLayer(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.layer3_relu3 = ReluLayer()
        self.layer3_maxpool = MaxPoolLayer(kernel_size=2, stride=2)

        self.layer4_conv1 = ConvolutionLayer(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.layer4_relu1 = ReluLayer()
        self.layer4_conv2 = ConvolutionLayer(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.layer4_relu2 = ReluLayer()
        self.layer4_conv3 = ConvolutionLayer(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.layer4_relu3 = ReluLayer()
        self.layer4_maxpool = MaxPoolLayer(kernel_size=2, stride=2)

        self.layer5_conv1 = ConvolutionLayer(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.layer5_relu1 = ReluLayer()
        self.layer5_conv2 = ConvolutionLayer(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.layer5_relu2 = ReluLayer()
        self.layer5_conv3 = ConvolutionLayer(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.layer5_relu3 = ReluLayer()
        self.layer5_maxpool = MaxPoolLayer(kernel_size=2, stride=2)

        self.flatten = FlattenLayer()
        self.fullyconnect1 = FullyConnectLayer(in_features=512 * 7 * 7, out_features=4096)
        self.relu_1 = ReluLayer()
        self.fullyconnect2 = FullyConnectLayer(in_features=4096, out_features=4096)
        self.relu_2 = ReluLayer()
        self.fullyconnect3 = FullyConnectLayer(in_features=4096, out_features=num_classes)

        self.graph_layers = None
        self.create_graph()

    def create_graph(self):
        self.graph_layers = {
            'layer1_conv1': self.layer1_conv1, 'layer1_relu1': self.layer1_relu1,
            'layer1_conv2': self.layer1_conv2, 'layer1_relu2': self.layer1_relu2,
            'layer1_maxpool': self.layer1_maxpool,

            'layer2_conv1': self.layer2_conv1, 'layer2_relu1': self.layer2_relu1,
            'layer2_conv2': self.layer2_conv2, 'layer2_relu2': self.layer2_relu2,
            'layer2_maxpool': self.layer2_maxpool,

            'layer3_conv1': self.layer3_conv1, 'layer3_relu1': self.layer3_relu1,
            'layer3_conv2': self.layer3_conv2, 'layer3_relu2': self.layer3_relu2,
            'layer3_conv3': self.layer3_conv3, 'layer3_relu3': self.layer3_relu3,
            'layer3_maxpool': self.layer3_maxpool,

            'layer4_conv1': self.layer4_conv1, 'layer4_relu1': self.layer4_relu1,
            'layer4_conv2': self.layer4_conv2, 'layer4_relu2': self.layer4_relu2,
            'layer4_conv3': self.layer4_conv3, 'layer4_relu3': self.layer4_relu3,
            'layer4_maxpool': self.layer4_maxpool,

            'layer5_conv1': self.layer5_conv1, 'layer5_relu1': self.layer5_relu1,
            'layer5_conv2': self.layer5_conv2, 'layer5_relu2': self.layer5_relu2,
            'layer5_conv3': self.layer5_conv3, 'layer5_relu3': self.layer5_relu3,
            'layer5_maxpool': self.layer5_maxpool,

            'flatten': self.flatten,
            'fullyconnect1': self.fullyconnect1, 'relu1': self.relu_1,
            'fullyconnect2': self.fullyconnect2, 'relu2': self.relu_2,
            'fullyconnect3': self.fullyconnect3,
        }

    def forward(self, x):
        for name in self.graph_layers.keys():
            # 正向传播的同时，打印均值和总和，用于核对执行过程是否正确
            print(f'forward: {name}: {x.mean()} {x.sum()}')
            x = self.graph_layers[name].forward(x)
        return x

    def backward(self, grad):
        for name in reversed(list(self.graph_layers.keys())):
            # 反向传播的同时，打印均值和总和，用于核对执行过程是否正确
            print(f'backward: {name}: {grad.mean()} {grad.sum()}')
            grad = self.graph_layers[name].backward(grad)
        return grad

    def resume_weights(self, ckpt):
        for name, params in ckpt.items():
            self.graph_layers[name].load_params(params['weight'], params['bias'])
        print('reloaded success')

In [7]:
# 导入预处理函数
from main import process_image

# 分类类别
CLASSES = ('daisy', 'roses', 'sunflowers', 'tulips')

# 网络初始化、加载权重参数
model = VGG16(4)
ckpt = np.load('./file/vgg16_ckpt.npy', allow_pickle=True).item()
model.resume_weights(ckpt)

start_time = time.time()

# 输入图片预处理
image_path = './file/tulips_demo.jpg'
tensor = process_image(image_path)

# 模型正向传播
outputs = model.forward(tensor)
print(f'forward outputs: {outputs}')
pred = int(np.argmax(outputs))
print(f'predict class: {CLASSES[pred]}')

# 计算loss
label = np.array([1, ])
loss_func = CrossEntropy()
loss = loss_func.forward(outputs, label)
print(f'loss: {loss}')

# 反向传播
grad = loss_func.backward()
grad = model.backward(grad)

end_time = time.time()
print(f'current task cost time: {end_time - start_time}')

reloaded success
forward: layer1_conv1: -0.06873162090778351 -10346.033203125
forward: layer1_relu1: -0.05778738483786583 -185570.546875
forward: layer1_conv2: 0.28507915139198303 915464.4375
forward: layer1_relu2: -0.013939479365944862 -44763.34765625
forward: layer1_maxpool: 0.33006376028060913 1059921.875
forward: layer2_conv1: 0.4298982322216034 345129.1875
forward: layer2_relu1: 0.017751257866621017 28501.98828125
forward: layer2_conv2: 0.3358146846294403 539194.8125
forward: layer2_relu2: 0.029406482353806496 47215.98828125
forward: layer2_maxpool: 0.34632712602615356 556073.9375
forward: layer3_conv1: 0.5311101675033569 213191.875
forward: layer3_relu1: 0.003264831844717264 2621.059326171875
forward: layer3_conv2: 0.3423604369163513 274852.4375
forward: layer3_relu2: 0.018941137939691544 15206.248046875
forward: layer3_conv3: 0.3454575836658478 277338.875
forward: layer3_relu3: -0.013854059390723705 -11122.2607421875
forward: layer3_maxpool: 0.32738643884658813 262831.0625
forwa